In [ ]:
import requests
import json
from pprint import pprint

import pandas as pd
import numpy as np
import glob

from tqdm.notebook import tqdm

from os import makedirs
from os.path import join, exists
from datetime import date, timedelta

In [ ]:
apikey= YOUR_KEY

In [ ]:
articles_directory = join('guardian_data', 'articles')
makedirs(articles_directory, exist_ok=True)

api_key = your_api_key
api_endpoint = 'http://content.guardianapis.com/search'
parameters = {
    'from-date': "",
    'to-date': "",
    'order-by': "newest",
    'show-fields': 'all',
    'page-size': 200,
    'api-key': api_key,
    'show-elements': 'image'
}

start_date = date(2006, 1, 1)
end_date = date(2020,4, 1)
dayrange = range((end_date - start_date).days + 1)

for daycount in tqdm(dayrange):
    dt = start_date + timedelta(days=daycount)
    datestr = dt.strftime('%Y-%m-%d')
    fname = join(articles_directory, datestr + '.json')

    if not exists(fname):
        print("Downloading", datestr)
        all_results = []
        parameters['from-date'] = datestr
        parameters['to-date'] = datestr
        current_page = 1
        total_pages = 200
        
        while current_page <= total_pages:
            print("...page", current_page)
            parameters['page'] = current_page
            req = requests.get(api_endpoint, parameters)
            df = req.json()
            all_results.extend(df['response']['results'])
            current_page += 1
            total_pages = df['response']['pages']

        with open(fname, 'w') as f:
            print("Writing to", fname)

            f.write(json.dumps(all_results, indent=2))

In [ ]:
path_to_json = '/Users/mariia/Docs no synch/nonverb/Python/tempdata/articles/' 
temp = pd.DataFrame()

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

dfs = [] 
for file in file_list:
    data = pd.read_json(file) 
    dfs.append(data) 

temp = pd.concat(dfs, ignore_index=True)

In [ ]:
def process_json(df):
    d = df['fields'].apply(pd.Series)
    df_out = pd.concat([df, d], axis=1)
    df_out=df_out[['id', 'sectionName', 'webPublicationDate', 'webTitle',\
                   'shortUrl', 'pillarName', 'headline', 'standfirst', 'body', \
                   'wordcount', 'bodyText', 'main', 'thumbnail']]
    df_out['main'] = df_out.main.str.extract(r'src\s*=\s*\"(.+?)\"')
    df_out.rename(columns={"webPublicationDate": "date", "standfirst": "snippet", \
                           "shortUrl": "n_url", "wordcount": "word_count", "sectionName": "section", \
                           "bodyText": "article_text", "thumbnail": "image", "main": "images"}, inplace = True)
    df_out['date']=pd.to_datetime(df_out['date']).dt.strftime('%Y-%m-%d')
    return df_out

In [ ]:
df = process_json(temp)
df